Needed imports and utility

In [ ]:
import os
import ansys.speos.core as core
from ansys.api.speos.sensor.v1 import camera_sensor_pb2

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

def print_message(msg):
    print(core.protobuf_message_to_str(msg))

# Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50051)
clean_all_dbs(speos.client)

# Create empty scene and load file

In [ ]:
speos_file = os.path.join(tests_data_path, os.path.join("Inverse_SeveralSensors.speos", "Inverse_SeveralSensors.speos"))

my_scene = speos.client.scenes().create()
my_scene.load_file(file_uri=speos_file)

# Print data models
## Print whole my_scene datamodel

In [ ]:
print(my_scene)

## Look at all sensors in my_scene

In [ ]:
for sensor_i in my_scene.get().sensors:
    print(sensor_i)  # Print instance data model
    print(speos.client.get_item(key=sensor_i.sensor_guid))  # Print template data model

## Look at camera sensors in my_scene

In [ ]:
for sensor_i in my_scene.get().sensors:
    if sensor_i.HasField("camera_properties"):
        print(sensor_i) # Print instance data model
        print(speos.client.get_item(key=sensor_i.sensor_guid))  # Print template data model

# Equivalent by creating a camera instances list
#camera_instances = [sensor_i for sensor_i in my_scene.get().sensors if sensor_i.HasField("camera_properties")]
#for camera_i in camera_instances:
#    print(camera_i)  # Print instance data model
#    print(speos.client.get_item(key=sensor_i.sensor_guid))  # Print template data model

# Modify instance of the camera

Two identical examples are given:
- one using directly protobuf message
- second using core factories

Please keep in mind that core factories are more intended to create protobuf messages from scratch.  
So using them to modify existing message is not the primary use case.  
It will force the user to modify the whole object.  
Whereas using directly protobuf message, user can modify specific part of it and keep the rest as is.

## Using directly protobuf message

In [ ]:
my_scene_data = my_scene.get() # get() = retrieve datamodel corresponding to my_scene from database
camera_i_0 = my_scene_data.sensors[0]
assert camera_i_0.HasField("camera_properties")

# Modification on protobuf message
camera_i_0.camera_properties.ClearField("axis_system")
camera_i_0.camera_properties.axis_system.extend([17.0, 10.0, 15.0] + [0.0, 0.0, -1.0] + [0.0, 1.0, 0.0] + [1.0, 0.0, 0.0])
camera_i_0.camera_properties.layer_type_none.SetInParent()
my_scene.set(my_scene_data) # Update using modified datamodel

print(my_scene.get().sensors[0]) # Do another get() to check new value on database

## Using core factories

In [ ]:
my_scene_data = my_scene.get() # get() = retrieve datamodel corresponding to my_scene from database
camera_i_0 = my_scene_data.sensors[0]
assert camera_i_0.HasField("camera_properties")

# Using factories (layer type none is set by default)
# In this case the whole message camera_properties is replaced by the return of core.SceneFactory.camera_sensor_props
camera_i_0.camera_properties.CopyFrom(
    core.SceneFactory.camera_sensor_props(
        axis_system=core.AxisSystem(
            origin=[17.0, 10.0, 15.0], 
            x_vect=[0.0, 0.0, -1.0], 
            y_vect=[0.0, 1.0, 0.0], 
            z_vect=[1.0, 0.0, 0.0])))
my_scene.set(my_scene_data)

print(my_scene.get().sensors[0]) # Do another get() to check new value on database

# Modify template of the camera

Two identical examples are given:
- one using directly protobuf message
- second using core factories

Please keep in mind that core factories are more intended to create protobuf messages from scratch.  
So using them to modify existing message is not the primary use case.  
It will force the user to modify the whole object.  
Whereas using directly protobuf message, user can modify specific part of it and keep the rest as is.

## Using directly protobuf message

In [ ]:
new_distortion_file = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraDistortion_150deg.OPTDistortion"))

# Retrieve SensorTemplateLink corresponding to camera_i_0.sensor_guid
camera_t_0 = speos.client.get_item(camera_i_0.sensor_guid)
assert isinstance(camera_t_0, core.SensorTemplateLink)

# get() = retrieve datamodel corresponding to camera_t_0 from database
camera_t_0_data = camera_t_0.get()

# Modification on protobuf message
assert camera_t_0_data.HasField("camera_sensor_template")
camera_t_0_data.camera_sensor_template.distorsion_file_uri = new_distortion_file
camera_t_0_data.camera_sensor_template.focal_length = 4.5
assert camera_t_0_data.camera_sensor_template.HasField("sensor_mode_photometric")
camera_t_0_data.camera_sensor_template.sensor_mode_photometric.gamma_correction = 2.5

# Update using modified datamodel
camera_t_0.set(camera_t_0_data)

print(camera_t_0) # Print ObjectLing to see its datamodel in database

## Using core factories

In [ ]:
new_distortion_file = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraDistortion_150deg.OPTDistortion"))

# Retrieve SensorTemplateLink corresponding to camera_i_0.sensor_guid
camera_t_0 = speos.client.get_item(camera_i_0.sensor_guid)
assert isinstance(camera_t_0, core.SensorTemplateLink)

# get() = retrieve datamodel corresponding to camera_t_0 from database
camera_t_0_data = camera_t_0.get()

# As precised this is an example but not the primary aim for SensorTemplateFactory.
# Indeed as the message will be copied entirely, it is needed here to give also data that we don't want to change (taken from camera_t_0_data)
camera_t_0_data.CopyFrom(
    core.SensorTemplateFactory.camera(
        name=camera_t_0_data.name, 
        distortion_file_uri=new_distortion_file, # Modify distortion file
        transmittance_file_uri=camera_t_0_data.camera_sensor_template.sensor_mode_photometric.transmittance_file_uri,
        spectrum_file_uris=[camera_t_0_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.red_spectrum_file_uri,
                            camera_t_0_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.green_spectrum_file_uri,
                            camera_t_0_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.blue_spectrum_file_uri],
        settings=core.SensorTemplateFactory.CameraSettings(gamma_correction=2.5, # Modify gamma_correction
                                                           focal_length=4.5, # Modify focal_length
                                                           imager_distance=camera_t_0_data.camera_sensor_template.imager_distance,
                                                           f_number=camera_t_0_data.camera_sensor_template.f_number),
        dimensions=core.SensorTemplateFactory.CameraDimensions(horz_pixel=camera_t_0_data.camera_sensor_template.horz_pixel,
                                                               vert_pixel=camera_t_0_data.camera_sensor_template.vert_pixel,
                                                               width=camera_t_0_data.camera_sensor_template.width,
                                                               height=camera_t_0_data.camera_sensor_template.height),
        wavelengths_range=core.SensorTemplateFactory.WavelengthsRange(start=camera_t_0_data.camera_sensor_template.sensor_mode_photometric.wavelengths_range.w_start,
                                                                      end=camera_t_0_data.camera_sensor_template.sensor_mode_photometric.wavelengths_range.w_end,
                                                                      sampling=camera_t_0_data.camera_sensor_template.sensor_mode_photometric.wavelengths_range.w_sampling)))

# Update using modified datamodel
camera_t_0.set(camera_t_0_data)

print(camera_t_0) # Print ObjectLing to see its datamodel in database

# Create camera template

Two identical examples are given:
- one using directly protobuf message
- second using core factories

In this case it can be interesting to use factory.
Indeed some default values are present, that can allow to fill less parameters.

## Using directly protobuf message

In [ ]:
sensor_t_db = speos.client.sensor_templates()

# Create protobuf message SensorTemplate
sensor_t_data = core.SensorTemplate(name="Camera_default by protobuf message")
sensor_t_data.camera_sensor_template.sensor_mode_photometric.acquisition_integration = 0.01
sensor_t_data.camera_sensor_template.sensor_mode_photometric.acquisition_lag_time = 0
sensor_t_data.camera_sensor_template.sensor_mode_photometric.transmittance_file_uri = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraTransmittance.spectrum"))
sensor_t_data.camera_sensor_template.sensor_mode_photometric.gamma_correction = 2.2
sensor_t_data.camera_sensor_template.sensor_mode_photometric.png_bits = camera_sensor_pb2.EnumSensorCameraPNGBits.PNG_16
sensor_t_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.red_spectrum_file_uri = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityRed.spectrum"))
sensor_t_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.green_spectrum_file_uri = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityGreen.spectrum"))
sensor_t_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.blue_spectrum_file_uri = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityBlue.spectrum"))
sensor_t_data.camera_sensor_template.sensor_mode_photometric.color_mode_color.balance_mode_none.SetInParent()
sensor_t_data.camera_sensor_template.sensor_mode_photometric.wavelengths_range.w_start = 400
sensor_t_data.camera_sensor_template.sensor_mode_photometric.wavelengths_range.w_end = 700
sensor_t_data.camera_sensor_template.sensor_mode_photometric.wavelengths_range.w_sampling = 13
sensor_t_data.camera_sensor_template.focal_length = 5
sensor_t_data.camera_sensor_template.imager_distance = 10
sensor_t_data.camera_sensor_template.f_number = 20
sensor_t_data.camera_sensor_template.distorsion_file_uri = os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraDistortion_130deg.OPTDistortion"))
sensor_t_data.camera_sensor_template.horz_pixel = 640
sensor_t_data.camera_sensor_template.vert_pixel = 480
sensor_t_data.camera_sensor_template.width = 5
sensor_t_data.camera_sensor_template.height = 5

# Store it in db and retrieve SensorTemplateLink
sensor_t_new = sensor_t_db.create(message=sensor_t_data)
print(sensor_t_new)

## Using core factories

In [ ]:
sensor_t_db = speos.client.sensor_templates()

# Create protobuf message SensorTemplate using factory default params
sensor_t_data_2 = core.SensorTemplateFactory.camera(name="Camera_default by factory",
                                                    distortion_file_uri=os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraDistortion_130deg.OPTDistortion")),
                                                    transmittance_file_uri=os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraTransmittance.spectrum")),
                                                    spectrum_file_uris=[os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityRed.spectrum")),
                                                                        os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityGreen.spectrum")),
                                                                        os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityBlue.spectrum"))]) 

# Store it in db and retrieve SensorTemplateLink
sensor_t_new_2 = sensor_t_db.create(message=sensor_t_data_2)
print(sensor_t_new_2)

In [ ]:
sensor_t_db = speos.client.sensor_templates()

# Create protobuf message SensorTemplate giving all params
sensor_t_data_3 = core.SensorTemplateFactory.camera(name="Camera_default by factory with params",
                                                    distortion_file_uri=os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraDistortion_130deg.OPTDistortion")),
                                                    transmittance_file_uri=os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraTransmittance.spectrum")),
                                                    spectrum_file_uris=[os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityRed.spectrum")),
                                                                        os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityGreen.spectrum")),
                                                                        os.path.join(tests_data_path, os.path.join("CameraInputFiles", "CameraSensitivityBlue.spectrum"))],
                                                    settings=core.SensorTemplateFactory.CameraSettings(gamma_correction=2.2,
                                                                                                       focal_length=5,
                                                                                                       imager_distance=10,
                                                                                                       f_number=20),
                                                    dimensions=core.SensorTemplateFactory.CameraDimensions(horz_pixel=640,
                                                                                                           vert_pixel=480,
                                                                                                           width=5,
                                                                                                           height=5),
                                                    wavelengths_range=core.SensorTemplateFactory.WavelengthsRange(start=400,
                                                                                                                  end=700,
                                                                                                                  sampling=13),
                                                    camera_balance_mode=None) 

# Store it in db and retrieve SensorTemplateLink
sensor_t_new_3 = sensor_t_db.create(message=sensor_t_data_3)

# The extremes have been presented : using all default or giving all parameters.
# For sure it can be mixed.

# Add a camera instance in the scene

In [ ]:
# Retrieve scene datamodel
my_scene_data = my_scene.get()

# Create camera instance
camera_i_2 = core.SceneFactory.sensor_instance(name=sensor_t_new_2.get().name + ".1",
                                               sensor_template=sensor_t_new_2,
                                               properties=core.SceneFactory.camera_sensor_props(axis_system=core.AxisSystem(origin=[50, 50, 50]),
                                                                                                layer_type=core.SceneFactory.Properties.Sensor.LayerType.Source()))
# Modify scene datamodel to add camera + reference it in the first simulation
my_scene_data.sensors.append(camera_i_2)
my_scene_data.simulations[0].sensor_paths.append(camera_i_2.name)

# Update value in db
my_scene.set(my_scene_data)

# Check scene data after update
print(my_scene)

# List all templates

In [ ]:
# Action on all SensorTemplateLinks
for sensor_t in speos.client.sensor_templates().list():
    print(sensor_t.key)
    print(sensor_t)
    print("")

In [ ]:
# Action on SensorTemplateLinks that are cameras
for sensor_t in speos.client.sensor_templates().list():
    if sensor_t.get().HasField("camera_sensor_template"):
        print(sensor_t.key)
        print(sensor_t)
        print("")

In [ ]:
# Action on SensorTemplateLinks with specific name
for sensor_t in speos.client.sensor_templates().list():
    if sensor_t.get().name == "Camera_default by factory":
        print(sensor_t.key)
        print(sensor_t)
        break